# Notebook 1
Raw data export into np.memaps.

### Load important modules

In [1]:
# Standard modules
import numpy as np
import os
import logging

import pickle

# Move to root directory for easier module handling
os.chdir("../..")
print(os.listdir("."))
from notebooks.data_processing.modules.maldi_conversion import process_raw_data, extract_raw_data
from notebooks.data_processing.modules.lookup_tables import process_lookup_tables
from modules.tools.misc import delete_all_files_in_folder

# multithreading/multiprocessing
from multiprocessing import Pool
from threadpoolctl import threadpool_limits

# set thread limit
threadpool_limits(16)


['data', 'pages', 'app.py', 'assets', 'config.py', 'index.py', 'main.py', 'TODO.py', 'notebooks', 'modules', '__pycache__']


### Create a list of raw data filenames

In [2]:
path_brain_1 =  "/data/lipidatlas/data/data_raw/"
path_brain_2 =  "/data/lipidatlas/data/data_raw/BRAIN2/"
path_brain_1_temp = "notebooks/data_processing/data/temp/brain_1"
path_brain_2_temp = "notebooks/data_processing/data/temp/brain_2"
split_value_1 = "MouseBrainCMC_S"
split_value_2 = "MouseBrain2_S"
ll_t_names = []
for path_brain, path_brain_temp, split_value in zip([path_brain_1, path_brain_2], [path_brain_1_temp, path_brain_2_temp], [split_value_1,split_value_2]):
    # Load filenames
    l_t_names = sorted(
        [
            [
                int(name.split(split_value)[1].split("_")[0].split("A")[0].split("(")[0]),
                path_brain + name + "/" + name,
            ]
            for name in os.listdir(path_brain)
            if "MouseBrain" in name
        ]
    )

    # Correct for duplicates
    for t_names_1, t_names_2 in zip(l_t_names[:-1], l_t_names[1:]):
        if t_names_2[0] == t_names_1[0]:
            t_names_2.append("bis")
            print("WARNING: duplicate for slice " + str(t_names_1[0]))

    # Remove slices that have already been processed


    os.makedirs(path_brain_temp, exist_ok=True)
    remove_already_loaded = False
    if remove_already_loaded:
        existing_names = [int(name.split("_")[1][:-7]) for name in os.listdir(path_brain_temp) if "raw" in name]
        l_t_names = [x for x in l_t_names if x[0] not in existing_names]

    # Print the final list of names
    for t_names in l_t_names:
        print(t_names[0], t_names[1].split("/")[-1])

    ll_t_names.append(l_t_names)


1 20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um
2 20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um
3 20210213_MouseBrainCMC_S3AC4_2Dpixelmode_371x195_Att25_25um
4 20210214_MouseBrainCMC_S4AD3_2Dpixelmode_354x228_Att25_25um
5 20210218_MouseBrainCMC_S5AE3_2Dpixelmode_396x272_Att25_25um
6 20210219_MouseBrainCMC_S6AE3_2Dpixelmode_423x282_Att25_25um
7 20210220_MouseBrainCMC_S7AF5_2Dpixelmode_427x263_Att25_25um
8 20210531_MouseBrainCMC_S8_duplicate_2Dpixelmode_430x285_Att30_25um
9 20210224_MouseBrainCMC_S9AH4_2Dpixelmode_467x278_Att25_25um
10 20210210_MouseBrainCMC_S10(brain2_20)_394x282_Att30_25um
11 20210301_MouseBrainCMC_S11AK5_2Dpixelmode_448x277_Att25_25um
12 20210303_MouseBrainCMC_S12AL1_2Dpixelmode_393x266_Att25_25um
13 20210304_MouseBrainCMC_S13AM1_2Dpixelmode_413x310_Att25_25um
14 20210305_MouseBrainCMC_S14AN1_2Dpixelmode_409x285_Att25_25um
15 20210313_MouseBrainCMC_S15AO2_2Dpixelmode_451x292_Att25_25um
16 20210530_MouseBrainCMC_S16_duplicate_2Dpixelmode_454

In [3]:
brain_1 = True
if brain_1:
    l_t_names = ll_t_names[0]
else:
    l_t_names = ll_t_names[1]
print(l_t_names )

[[1, '/data/lipidatlas/data/data_raw/20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um/20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um'], [2, '/data/lipidatlas/data/data_raw/20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um/20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um'], [3, '/data/lipidatlas/data/data_raw/20210213_MouseBrainCMC_S3AC4_2Dpixelmode_371x195_Att25_25um/20210213_MouseBrainCMC_S3AC4_2Dpixelmode_371x195_Att25_25um'], [4, '/data/lipidatlas/data/data_raw/20210214_MouseBrainCMC_S4AD3_2Dpixelmode_354x228_Att25_25um/20210214_MouseBrainCMC_S4AD3_2Dpixelmode_354x228_Att25_25um'], [5, '/data/lipidatlas/data/data_raw/20210218_MouseBrainCMC_S5AE3_2Dpixelmode_396x272_Att25_25um/20210218_MouseBrainCMC_S5AE3_2Dpixelmode_396x272_Att25_25um'], [6, '/data/lipidatlas/data/data_raw/20210219_MouseBrainCMC_S6AE3_2Dpixelmode_423x282_Att25_25um/20210219_MouseBrainCMC_S6AE3_2Dpixelmode_423x282_Att25_25um'], [7, '/data/lipidatlas/data/data_raw/20210220_


### Extract raw data into numpy arrays with multiprocessing

In [4]:
if False:
    multiprocessing = False
    if multiprocessing:
        with Pool(processes=14) as pool:
            [x for x in pool.imap_unordered(extract_raw_data, l_t_names)]
    else:
        # Normal (single-processed) map
        [x for x in map(extract_raw_data, l_t_names)]

### Process raw data into numpy arrays with multiprocessing

In [5]:
multiprocessing = False
if multiprocessing:
    with Pool(processes=16) as pool:
        [x for x in pool.imap_unordered(process_raw_data, l_t_names)]
else:
    # Normal (single-processed) map
    [x for x in map(process_raw_data, l_t_names)]


Compute and normalize pixels values according to TIC
Filtering out noise and matrix peaks
Prepare data for standardization
Standardize data
74358 have been transformed, with an average of  0.0030259017187121763 peaks transformed
Sorting by m/z value for averaging


/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:558: RuntimeWarning: divide by zero encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)
/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:558: RuntimeWarning: invalid value encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)


Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Sorting by m/z value for averaging before standardization
Getting spectrums array averaged accross pixels
Double sorting according to pixel and mz high-res array
Getting corresponding spectra arrays
Saving : /data/lipidatlas/data/data_raw/20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um/20210210_MouseBrainCMC_S1AA1_2Dpixelmode_322x231_Att25_25um
Compute and normalize pixels values according to TIC
Filtering out noise and matrix peaks
Prepare data for standardization
Standardize data
There seems to be a problem with the computation of the integral
0.0014067147135542308 0.0006830153288319707
26 27
78699 have been transformed, with an average of  0.0031766604404122037 peaks transformed
Sorting by m/z value for averaging


/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:558: RuntimeWarning: divide by zero encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)
/home/colas/lbae/lbae/notebooks/data_processing/modules/maldi_conversion.py:558: RuntimeWarning: invalid value encountered in true_divide
  array_corrective_factors = np.nan_to_num(arrays_after_transfo / arrays_before_transfo)


Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Sorting by m/z value for averaging before standardization
Getting spectrums array averaged accross pixels
Double sorting according to pixel and mz high-res array
Getting corresponding spectra arrays
Saving : /data/lipidatlas/data/data_raw/20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um/20210211_MouseBrainCMC_S2AB5_2Dpixelmode_370x214_Att25_25um
Compute and normalize pixels values according to TIC
Filtering out noise and matrix peaks
Prepare data for standardization
Standardize data
72320 have been transformed, with an average of  0.00031803097345132743 peaks transformed
Sorting by m/z value for averaging
Getting spectrums array averaged accross pixels
Build the low-resolution averaged array from the high resolution averaged array
Sorting by m/z value for averaging before standardization
Getting spectrums array averaged accross pixels
Double sorting

### Build lookup tables

In [4]:
multiprocessing = True
if multiprocessing:
    # Multiprocessing
    with Pool(processes=16) as pool:
        [x for x in pool.map(process_lookup_tables, l_t_names)]
else:
    # Normal (single-processed) map
    [x for x in map(process_lookup_tables, l_t_names)]



Size (in mb) of lookup_table_spectra_high_res:  551.95
Shape of lookup_table_spectra_high_res:  (2000, 72345)
Size (in mb) of lookup_table_spectra_high_res:  604.1
Shape of lookup_table_spectra_high_res:  (2000, 79180)
Size (in mb) of lookup_table_spectra_high_res:  615.78
Shape of lookup_table_spectra_high_res:  (2000, 80712)
Size (in mb) of lookup_table_spectra_high_res:  567.49
Shape of lookup_table_spectra_high_res:  (2000, 74382)
Size (in mb) of lookup_table_spectra_high_res:  821.78
Shape of lookup_table_spectra_high_res:  (2000, 107712)
Size (in mb) of lookup_table_spectra_high_res:  990.49
Shape of lookup_table_spectra_high_res:  (2000, 129826)
Size (in mb) of lookup_table_spectra_high_res:  910.08
Shape of lookup_table_spectra_high_res:  (2000, 119286)
Size (in mb) of lookup_table_spectra_high_res:  934.98
Shape of lookup_table_spectra_high_res:  (2000, 122550)
Size (in mb) of lookup_table_spectra_high_res:  797.56
Shape of lookup_table_spectra_high_res:  (2000, 104538)
Size (

### Record everything and clean 

Record everything in memap files and a pickled dictonnary

In [ ]:
# ! Change all slices indexes of brain 2 such that they can be in the same output folder as brain 1 

In [7]:
output_folder = "data/whole_dataset/"
if brain_1:
    input_folder = "notebooks/data_processing/data/temp/brain_1/"
else:
    input_folder = "notebooks/data_processing/data/temp/brain_2/"
        
os.makedirs(output_folder, exist_ok=True)

dic_slices = {}
# Loop over slice files
for slice_name in os.listdir(input_folder):
    if 'raw' in slice_name or 'checkpoints' in slice_name:
        continue

    print(slice_name)
    # Extract slice index
    slice_index = int(slice_name.split("_")[1][:-4])

    # Load slice arrays
    npzfile = np.load(input_folder + slice_name)
    array_pixel_indexes_high_res = npzfile["array_pixel_indexes_high_res"]
    array_spectra_high_res = npzfile["array_spectra_high_res"]
    array_averaged_mz_intensity_low_res = npzfile["array_averaged_mz_intensity_low_res"]
    array_averaged_mz_intensity_high_res = npzfile["array_averaged_mz_intensity_high_res"]
    array_averaged_mz_intensity_high_res_after_standardization = npzfile["array_averaged_mz_intensity_high_res_after_standardization"]
    image_shape = npzfile["image_shape"]
    divider_lookup = npzfile["divider_lookup"]
    lookup_table_spectra_high_res = npzfile["lookup_table_spectra_high_res"]
    cumulated_image_lookup_table_high_res = npzfile["cumulated_image_lookup_table_high_res"]
    lookup_table_averaged_spectrum_high_res = npzfile["lookup_table_averaged_spectrum_high_res"]
    array_peaks_corrected = npzfile["array_peaks_corrected"]
    array_corrective_factors = npzfile["array_corrective_factors"]

    # Print array size
    # print size used by each array in mb
    print(round(array_pixel_indexes_high_res.nbytes / 1024 / 1024, 2))
    print(round(array_spectra_high_res.nbytes / 1024 / 1024, 2))
    print(round(array_averaged_mz_intensity_low_res.nbytes / 1024 / 1024, 2))
    print(round(array_averaged_mz_intensity_high_res.nbytes / 1024 / 1024, 2))
    print(round(lookup_table_spectra_high_res.nbytes / 1024 / 1024, 2))
    print(round(cumulated_image_lookup_table_high_res.nbytes / 1024 / 1024, 2))
    print(round(lookup_table_averaged_spectrum_high_res.nbytes / 1024 / 1024, 2))

    # Register the lightweights files in a pickled dictionnary
    dic_slices[slice_index] = {
        "image_shape": image_shape,
        "divider_lookup": divider_lookup,
        "array_avg_spectrum_downsampled": array_averaged_mz_intensity_low_res,
        "array_lookup_pixels": array_pixel_indexes_high_res,
        "array_lookup_mz_avg": lookup_table_averaged_spectrum_high_res,
        "array_peaks_transformed_lipids": array_peaks_corrected,
        "array_corrective_factors": array_corrective_factors,
    }

    # Build a memap for each of the heavier files to save RAM, save the corresponding shape in the
    # pickled dictionnary
    fp = np.memmap(
        output_folder + "array_spectra_" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=array_spectra_high_res.shape,
    )
    fp[:] = array_spectra_high_res[:]
    fp.flush()
    dic_slices[slice_index]["array_spectra_shape"] = array_spectra_high_res.shape

    fp = np.memmap(
        output_folder + "array_avg_spectrum_" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=array_averaged_mz_intensity_high_res.shape,
    )
    fp[:] = array_averaged_mz_intensity_high_res[:]
    fp.flush()
    dic_slices[slice_index]["array_avg_spectrum_shape"] = array_averaged_mz_intensity_high_res.shape

    fp = np.memmap(
        output_folder + "array_avg_spectrum_after_standardization_" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=array_averaged_mz_intensity_high_res_after_standardization.shape,
    )
    fp[:] = array_averaged_mz_intensity_high_res_after_standardization[:]
    fp.flush()
    dic_slices[slice_index]["array_avg_spectrum_after_standardization_shape"] = array_averaged_mz_intensity_high_res_after_standardization.shape


    fp = np.memmap(
        output_folder + "array_lookup_mz_" + str(slice_index) + ".mmap",
        dtype="int32",
        mode="w+",
        shape=lookup_table_spectra_high_res.shape,
    )
    fp[:] = lookup_table_spectra_high_res[:]
    fp.flush()
    dic_slices[slice_index]["array_lookup_mz_shape"] = lookup_table_spectra_high_res.shape

    fp = np.memmap(
        output_folder + "array_cumulated_lookup_mz_image_" + str(slice_index) + ".mmap",
        dtype="float32",
        mode="w+",
        shape=cumulated_image_lookup_table_high_res.shape,
    )
    fp[:] = cumulated_image_lookup_table_high_res[:]
    fp.flush()
    dic_slices[slice_index][
        "array_cumulated_lookup_mz_image_shape"
    ] = cumulated_image_lookup_table_high_res.shape

# Pickle the dict of lightweight data
with open(output_folder + "light_arrays.pickle", "wb") as handle:
    pickle.dump(dic_slices, handle)


slice_1.npz
0.57
148.1
0.05
0.98
567.49
567.49
0.01
slice_2.npz
0.6
158.53
0.06
1.1
604.1
604.1
0.01
slice_3.npz
0.55
71.81
0.05
0.94
551.95
551.95
0.01
slice_4.npz
0.62
61.66
0.05
1.32
615.78
615.78
0.01
slice_5.npz
0.82
136.45
0.05
1.13
821.78
821.78
0.01
slice_6.npz
0.91
210.06
0.06
1.32
910.08
910.08
0.01
slice_7.npz
0.86
218.57
0.06
1.24
856.79
856.79
0.01
slice_8.npz
0.93
162.56
0.06
1.2
934.98
934.98
0.01
slice_9.npz
0.99
270.59
0.06
1.36
990.49
990.49
0.01
slice_10.npz
0.85
273.05
0.06
1.35
847.69
847.69
0.01
slice_11.npz
0.95
251.87
0.06
1.25
946.78
946.78
0.01
slice_12.npz
0.8
226.24
0.06
1.21
797.56
797.56
0.01
slice_13.npz
0.98
232.6
0.08
1.8
976.79
976.79
0.01
slice_14.npz
0.89
223.25
0.06
1.24
889.32
889.32
0.01
slice_15.npz
1.0
228.24
0.06
1.27
1004.73
1004.73
0.01
slice_16.npz
1.02
200.49
0.06
1.32
1021.8
1021.8
0.01
slice_17.npz
0.99
178.74
0.06
1.3
985.34
985.34
0.01
slice_18.npz
1.05
171.64
0.06
1.37
1052.35
1052.35
0.01
slice_19.npz
0.7
144.88
0.05
1.12
700.93
700.9

Clean temporary folder

In [ ]:
clean = False
if clean:
    delete_all_files_in_folder(input_folder)
